In [1]:
import pandas as pd
import numpy as np
import sklearn 
import statsmodels.api as sm
from datetime import date

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from stargazer.stargazer import Stargazer
from statsmodels.iolib.summary2 import summary_col

In [2]:
regr_data = pd.read_csv('uas_data_R2.csv')
regr_data = regr_data[regr_data['retired'] == 0]

### Chetty income-specific u/e data

In [3]:
# ue_data = pd.read_csv('Employment - County - Weekly.csv').dropna()
# ue_data['date'] = ue_data[['year', 'month']].astype(str).agg('-'.join, axis=1)
# ue_data['date'] = pd.to_datetime(ue_data['date'])

In [4]:
ue_data = pd.read_csv('Employment - County - Weekly.csv').dropna()

ue_data['date'] = ue_data[['year', 'month']].astype(str).agg('-'.join, axis=1)
ue_data['date'] = pd.to_datetime(ue_data['date'])

ue_data2 = ue_data[(ue_data['date'].dt.date > date(2020, 3, 1)) & (ue_data['date'].dt.date < date(2021, 8, 1))]
ue_data2.replace('.', None, inplace=True)
ue_data2 = ue_data2.apply(pd.to_numeric)

<ipython-input-4-dd5903460e05>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ue_data2.replace('.', None, inplace=True)


In [5]:
ue_data3 = ue_data2.groupby('countyfips').mean()

In [6]:
#60k+ is considered top 25% in Chetty whereas 60k+ is considered top ~40% in other sources? 

#Return income quartile based on USC data and Chetty data encoding
def findIncomeQuartile(income):
    if (income <= 7):
        return 1
    elif (income <= 10):
        return 2
    elif (income <= 12):
        return 3
    else:
        return 4
    
#Fill in u/e rate in regression data using Chetty data, return new updated dataframe. 
def inputeUe(regr_data, ue_data):
    ret = regr_data.copy(deep = True)
    quartiles_dict = {1: 'emp_incq1', 2:'emp_incq2', 3:'emp_incq3', 4:'emp_incq4'}
    
    count=0
    normal = 0
    #For each individual in USC survey data
    for index in regr_data.index:
        person_row = ret.loc[index, :]
        cntyFIPS = person_row['cnty_FIPS']
        quartile = findIncomeQuartile(person_row['hhincome'])
        
        #Find u/e rate corresponding to correct income, else use county-level u/e 
        if (cntyFIPS in ue_data.index):
            ue_row = ue_data.loc[cntyFIPS, :]
        else:
            count+=1
            continue
            
        ue_rate = np.nan
        if (pd.notnull(ue_row[quartiles_dict[quartile]])):
            ue_rate = ue_row[quartiles_dict[quartile]]
        elif (quartile <=2 and pd.notnull(ue_row['emp_incbelowmed'])):
            ue_rate = ue_row['emp_incbelowmed']
        elif (quartile >=3 and pd.notnull(ue_row['emp_incabovemed'])):
            ue_rate = ue_row['emp_incabovemed']
        elif (pd.notnull(ue_row['emp'])):
            ue_rate = ue_row['emp']
            normal+=1
        else:
            count+=1

        ret.loc[index, 'ue'] = ue_rate
            
    
    return ret, count, normal

In [7]:
retdata, count, normal = inputeUe(regr_data, ue_data3)

In [8]:
retdata['ue'] = pd.to_numeric(retdata['ue'])
retdata['ue'] = -retdata['ue']
retdata['ue'] = (retdata.ue - retdata.ue.mean())/(retdata.ue.max() - retdata.ue.min())

### BLS data

In [9]:
bls_ue = pd.read_csv('laucntycur14.csv').dropna()
jan_baseline = pd.read_csv('laucntycur14-jan20.csv').dropna()

In [10]:
for df in [bls_ue, jan_baseline]:
    zeros = {1:'00', 2:'0', 3:''}
    state_pt = df['StateFIPS'].astype(int).astype(str)
    cnty_pt = df['CntyFIPS'].astype(int).astype(str)
    for i in cnty_pt.index:
        elem = cnty_pt[i]
        cnty_pt[i] = zeros[len(elem)] + elem
    df['cnty_FIPS'] = state_pt + cnty_pt
    df['cnty_FIPS'] = df['cnty_FIPS'].astype(int)

In [11]:
jan_baseline2 = jan_baseline[['cnty_FIPS', 'ue rate (%)']].groupby('cnty_FIPS').mean()
jan_baseline2 = jan_baseline2.rename(columns={'ue rate (%)':'jan_ue'})

In [12]:
bls_ue['ue rate (%)'] = pd.to_numeric(bls_ue['ue rate (%)'], errors='coerce')
bls_ue2 = bls_ue[['cnty_FIPS', 'ue rate (%)']].groupby('cnty_FIPS').mean()

In [13]:
bls_ue3 = pd.merge(bls_ue2, jan_baseline2, on='cnty_FIPS', how='left')
bls_ue3['bls_ue'] = (bls_ue3['ue rate (%)'] - bls_ue3['jan_ue'])/bls_ue3['jan_ue']
bls_ue3['bls_ue'] = (bls_ue3.bls_ue - bls_ue3.bls_ue.mean())/(bls_ue3.bls_ue.max() - bls_ue3.bls_ue.min())

### Create df with chetty and bls u/e rate

In [14]:
regr_data_ue = pd.merge(retdata, bls_ue3.bls_ue, on = 'cnty_FIPS')

In [15]:
regr_data_ue['ue'].describe(percentiles=[.2, .5, .8])

count    2.425000e+03
mean    -2.197555e-18
std      7.512954e-02
min     -5.061650e-01
20%     -4.605660e-02
50%     -1.984992e-03
80%      4.285888e-02
max      4.938350e-01
Name: ue, dtype: float64

In [16]:
regr_data_ue2 = regr_data_ue[pd.notnull(regr_data_ue['ue'])]

regr_data_ue2['ue_bls_c'] = pd.cut(regr_data_ue2['bls_ue'], bins=[-np.inf, -0.041, 0.074,  0.21, np.inf], labels=['10', '50', '90', '100'])
regr_data_ue2['ue_chetty_c'] = pd.cut(regr_data_ue2['ue'], bins=[-np.inf, -0.075, -0.00355, 0.108, np.inf], labels=['10', '50', '90', '100'])



<ipython-input-16-7530ec00da64>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regr_data_ue2['ue_bls_c'] = pd.cut(regr_data_ue2['bls_ue'], bins=[-np.inf, -0.041, 0.074,  0.21, np.inf], labels=['10', '50', '90', '100'])
<ipython-input-16-7530ec00da64>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regr_data_ue2['ue_chetty_c'] = pd.cut(regr_data_ue2['ue'], bins=[-np.inf, -0.075, -0.00355, 0.108, np.inf], labels=['10', '50', '90', '100'])


In [17]:
regr_data_ue2 = pd.get_dummies(regr_data_ue2, columns = ['ue_chetty_c'])
regr_data_ue2 = pd.get_dummies(regr_data_ue2, columns = ['ue_bls_c'])

## Regressions with unemployment data

In [18]:
regr_data_ue2 

,Unnamed: 0,cnty_FIPS,uasid,cl014_240,cl014_250,cl014_348,immigrant_gen_0 Non-immigrant,immigrant_gen_1 First generation immigrant,immigrant_gen_2 Second generation immigrant,immigrant_gen_3 Third generation immigrant,...,ue,bls_ue,ue_chetty_c_10,ue_chetty_c_50,ue_chetty_c_90,ue_chetty_c_100,ue_bls_c_10,ue_bls_c_50,ue_bls_c_90,ue_bls_c_100
0,0,25013,140100007,0.0,0.0,0.0,0,0,1,0,...,0.017877,0.142988,0,0,1,0,0,0,1,0
1,2,25027,140100047,0.0,1.0,0.0,0,0,1,0,...,-0.005840,0.159940,0,1,0,0,0,0,1,0
2,3071,25027,180811882,0.0,0.0,0.0,0,0,0,1,...,-0.008520,0.159940,0,1,0,0,0,0,1,0
3,3,25017,140100079,0.0,0.0,0.0,1,0,0,0,...,0.108606,0.221279,0,0,0,1,0,0,0,1
4,4,25017,140100081,0.0,0.0,0.0,1,0,0,0,...,0.108606,0.221279,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,4007,37025,191104874,0.0,0.0,0.0,1,0,0,0,...,-0.033120,0.055919,0,1,0,0,0,1,0,0
2876,4045,37025,200300016,0.0,0.0,0.0,1,0,0,0,...,0.154141,0.055919,0,0,0,1,0,1,0,0
2877,4050,37025,200300026,1.0,1.0,0.0,0,0,1,0,...,-0.072556,0.055919,0,1,0,0,0,1,0,0
2878,4018,8097,191105434,0.0,0.0,0.0,1,0,0,0,...,0.087691,0.451647,0,0,1,0,0,0,0,1


In [19]:
regr_data_ue2['imm'] = (regr_data_ue2 ['immigrant_gen_1 First generation immigrant'] == 1).astype(int) | (regr_data_ue2 ['immigrant_gen_2 Second generation immigrant'] == 1).astype(int)
#(regr_data_ue2 ['immigrant_gen_1 First generation immigrant'] == 1).astype(int)

In [20]:
regr_data_ue2['imm*ue_ever'] = regr_data_ue2['imm'] * regr_data_ue2['ue_ever']

regr_data_ue2['imm*ue_chetty_c_100'] = regr_data_ue2['imm'] * regr_data_ue2['ue_chetty_c_100']
regr_data_ue2['imm*ue_chetty_c_10'] = regr_data_ue2['imm'] * regr_data_ue2['ue_chetty_c_10']

regr_data_ue2['low_ed'] = regr_data_ue2['educ_c_8'] + regr_data_ue2['educ_c_12']
regr_data_ue2['low_ed*ue_chetty_c_100'] = regr_data_ue2['low_ed'] * regr_data_ue2['ue_chetty_c_100']
regr_data_ue2['low_ed*ue_chetty_c_10'] = regr_data_ue2['low_ed'] * regr_data_ue2['ue_chetty_c_10']

In [31]:
y2 = regr_data_ue2['cl014_348']
y2 = y2.rename(' Change in education view, July2021')

X_frac_imm = regr_data_ue2[[
    'imm', 
#     'ue_chetty_c_10', 
#     'ue_chetty_c_50', 
#     'ue_chetty_c_90', 
    'ue_chetty_c_100', 
        'imm*ue_chetty_c_100',
#     'low_ed*ue_chetty_c_100',
        'educ_c_8',  'educ_c_12', 
#     'educ_c_14', 
    'educ_c_16', 'educ_c_18', 'educ_c_20', 
    'hhi_c_0','hhi_c_20', 
#     'hhi_c_40', 
    'hhi_c_60', 'hhi_c_80', 
        'case1', 'case2', 
#     'case3', 
    'case4', 'case5',
    'age1', 
    'age2', 
#     'age3', 
    'age4',
    'ue_ever',
    'imm*ue_ever'
                       ]]
X_frac_imm = sm.add_constant(X_frac_imm)
reg = sm.OLS(y2, X_frac_imm)
result = reg.fit(cov_type='cluster', cov_kwds={'groups': regr_data_ue2['cnty_FIPS']})

print(summary_col([result], stars=True).as_text())


                     Change in education view, July2021
-------------------------------------------------------
const               0.0189                             
                    (0.0388)                           
imm                 0.1195***                          
                    (0.0255)                           
ue_chetty_c_100     0.0857**                           
                    (0.0404)                           
imm*ue_chetty_c_100 -0.1043**                          
                    (0.0480)                           
educ_c_8            0.0018                             
                    (0.1705)                           
educ_c_12           -0.0581*                           
                    (0.0307)                           
educ_c_16           -0.0047                            
                    (0.0261)                           
educ_c_18           0.0055                             
                    (0.0345)                   

In [23]:
regr_data_ue2['imm*ue_bls_c_100'] = regr_data_ue2['imm'] * regr_data_ue2['ue_bls_c_100']
regr_data_ue2['low_ed*ue_bls_c_100'] = regr_data_ue2['low_ed'] * regr_data_ue2['ue_bls_c_100']

In [24]:
y3 = regr_data_ue2['cl014_348']
y3 = y3.rename(' Change in education view, July2021')

X_frac_imm = regr_data_ue2[[
    'imm', 
#     'imm*ue_bls_c_100',
    'ue_bls_c_10', 
#     'ue_bls_c_50', 
#     'ue_bls_c_90', 
    'ue_bls_c_100', 
        'imm*ue_bls_c_100', 
            'low_ed*ue_bls_c_100', 
        'educ_c_8',  'educ_c_12', 
#     'educ_c_14', 
    'educ_c_16', 'educ_c_18', 'educ_c_20', 
    'hhi_c_0','hhi_c_20', 
#     'hhi_c_40', 
    'hhi_c_60', 'hhi_c_80', 
        'case1', 'case2', 
#     'case3', 
    'case4', 'case5',
    'age1', 
    'age2', 
#     'age3', 
    'age4',
    'ue_ever',
    'imm*ue_ever'
                       ]]
X_frac_imm = sm.add_constant(X_frac_imm)
reg = sm.OLS(y3, X_frac_imm)
result = reg.fit(cov_type='cluster', cov_kwds={'groups': regr_data_ue2['cnty_FIPS']})

print(summary_col([result], stars=True).as_text())


                     Change in education view, July2021
-------------------------------------------------------
const               0.0285                             
                    (0.0392)                           
imm                 0.1144***                          
                    (0.0267)                           
ue_bls_c_10         -0.0191                            
                    (0.0530)                           
ue_bls_c_100        -0.0520                            
                    (0.0411)                           
imm*ue_bls_c_100    0.0085                             
                    (0.0613)                           
low_ed*ue_bls_c_100 -0.0680                            
                    (0.0865)                           
educ_c_8            0.0235                             
                    (0.1759)                           
educ_c_12           -0.0512                            
                    (0.0314)                   

In [25]:
#regr_data_ue2.to_csv('reg_data_R_ue.csv')